

### 使い方
- Peasy_train_2104to2302.xlsx：学習データ。に入電内容と分類の対応関係を決める。
- 案件一覧_202303.xlsx：分類対象のデータ。コールセンタからもらったファイルをそのまま配置する
- 上の「ランタイム」からすべてのセルを実行を行う
- output_peasy.csv に企業と入電内容が分類されたファイルが出力される



## モデルの学習

In [1]:
from matplotlib import dates as mdates
import os
import joblib
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# 認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score 
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

!pip install fugashi
!pip install ipadic
!pip install transformers
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.ensemble import RandomForestClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 10.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 91.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556721 sha256=3baa2742d876e2b5ee550fbc368bc51e3fbd099d74b07020130a5d57d75a808a
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 

In [2]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
checkpoint = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# checkpointからtokenizerとmodelをロード
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint).to(device)

def tensorize_texts(texts):
  tokenized_texts = tokenizer(texts, padding = True, truncation = True, max_length=256, return_tensors="pt").to(device)

  with torch.no_grad():
    hidden_texts = model(**tokenized_texts) #dim : [batch_size(nr_sentences), tokens, emb_dim]  
  cls_texts = hidden_texts.last_hidden_state[:,0,:]
  return cls_texts.to(device)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
path = "/content/drive/MyDrive/Colab Notebooks/data/Peasy_train_2104to2302.xlsx"
df_train = pd.read_excel(path, sheet_name='Sheet1')

In [4]:
df_train.shape

(698, 2)

In [5]:
texts = df_train["Text"] 
le = LabelEncoder()
df_train["Target_le"] = le.fit_transform(df_train["Target"])
targets = df_train["Target_le"]
input_train, input_test, output_train, output_test = train_test_split(texts, targets, test_size=0.3, random_state=None )
train = pd.DataFrame({"text" : input_train, "target" : output_train})
test = pd.DataFrame({"text" : input_test, "target" : output_test})

In [6]:
MAX_LEN = 256
class CustomData(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data["text"].values[idx]
        encoded = self.tokenizer(
            text, padding = "max_length", max_length = MAX_LEN, truncation = True)
        input_ids = torch.tensor(
            encoded["input_ids"], dtype = torch.long)
        attention_mask = torch.tensor(
            encoded["attention_mask"], dtype = torch.long)
        token_type_ids = torch.tensor(
            encoded["token_type_ids"], dtype = torch.long)
        
        label = self.data["target"].values[idx]
        label =  torch.tensor(label, dtype = torch.int64)
        return input_ids, attention_mask, token_type_ids, label
    
train_data = CustomData(train, tokenizer)
test_data = CustomData(test, tokenizer)
train_dl = DataLoader(train_data, batch_size = 16, shuffle = True, drop_last = True)
test_dl = DataLoader(test_data, batch_size = 32, shuffle = False, drop_last = False)

batch = next(iter(train_dl))

In [7]:
# Number of unique values
NUM_UNIQUE_VALUE = len(df_train["Target_le"].unique())

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        self.head = nn.Linear(in_features = 768, out_features = NUM_UNIQUE_VALUE) # クラス数
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids
        )
        pooler_output = outputs.pooler_output # 768次元の埋め込み表現
        logits = self.head(pooler_output)
        return logits

if os.path.exists("/content/drive/MyDrive/Colab Notebooks/data/model.pt"):
  # load data
  trained_model = torch.load("/content/drive/MyDrive/Colab Notebooks/data/model.pt")
else:
  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

  # モデル呼び出し
  trained_model = CustomModel().to(device)

  # 最適化手法
  optimizer = torch.optim.Adam(trained_model.parameters(), lr = 1e-4)

  # 損失関数
  criterion = nn.CrossEntropyLoss()

  # 学習ログを保存する変数
  history = {"train": [], "test": []}

  # epochの数だけ学習を繰り返す
  for epoch in range(5):

      # 学習
      trained_model.train()
      train_loss = 0
      for batch in tqdm(train_dl):
          optimizer.zero_grad()
          input_ids = batch[0].to(device)
          attention_mask = batch[1].to(device)
          token_type_ids = batch[2].to(device)
          label = batch[3].to(device)
          logits = trained_model(input_ids, attention_mask, token_type_ids)
          loss = criterion(logits, label) # 誤差計算
          loss.backward()                 # 誤差伝播
          optimizer.step()                # パラメータ更新
          train_loss += loss.item()
      train_loss /= len(train_dl)

      # 検証
      trained_model.eval()
      test_loss = 0
      with torch.no_grad(): # パラメータ更新をしない
          for batch in tqdm(test_dl):
              input_ids = batch[0].to(device)
              attention_mask = batch[1].to(device)
              token_type_ids = batch[2].to(device)
              label = batch[3].to(device)
              logits = trained_model(input_ids, attention_mask, token_type_ids)
              loss = criterion(logits, label)
              test_loss += loss.item()
      test_loss /= len(test_dl)

      # ログ保存
      history["train"].append(train_loss)
      history["test"].append(test_loss)

  plt.plot(history["train"], label = "train")
  plt.plot(history["test"], label = "test")
  plt.legend()
  plt.show()

  # save data
  torch.save(trained_model, "/content/drive/MyDrive/Colab Notebooks/data/model.pt")

In [8]:
preds = []
trained_model.eval()
loader = DataLoader(test_data)
with torch.no_grad():
    for batch in tqdm(test_dl):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        token_type_ids = batch[2].to(device)
        logits = trained_model(input_ids, attention_mask, token_type_ids)
        preds.append(nn.Softmax(dim = 1)(logits).cpu().numpy())
preds = np.concatenate(preds, axis = 0)
pred_labels = preds.argmax(axis = 1)
print("\n", accuracy_score(test["target"].values, pred_labels))

100%|██████████| 7/7 [00:06<00:00,  1.04it/s]


 0.8523809523809524


In [9]:
test["Original_category"] = le.inverse_transform(test["target"])
test["Estimated_category"] = le.inverse_transform(pred_labels)
test

,text,target,Original_category,Estimated_category
7,利用開始操作が出来ず、出庫してしまったとのこと。,24,利用開始ができない,利用開始ができない
351,11番利用。空車で入庫するも、利用開始が押せないとのこと。,24,利用開始ができない,利用開始ができない
541,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】利用時間超過駐車\t1\t\t\t\t\t,23,利用時間超過駐車,利用時間超過駐車
504,2番に入庫して利用開始操作を行ったところ、1番で利用開始操作されてしまったとのこと。このまま...,27,利用開始誤り,利用開始誤り
153,1番車室利用。いつもPeasyを利用しているが、本日利用開始操作を忘れそのまま出庫してしまっ...,26,利用開始押下忘れ,利用開始誤り
...,...,...,...,...
612,ｷｰﾌﾟし来場するも、不正ありの連絡あり。ｺﾝｿｰﾙにて、7：47ｷｰﾌﾟ開始・8：11入庫確認。,8,キープ車室への不正入庫,精算方法
602,利用方法についてのお問い合わせ。,20,利用方法の問い合わせ,利用方法の問い合わせ
452,1番車室へ入庫するも、利用開始操作ができないとの事。ｱﾌﾟﾘを開くもｻｰﾊﾞｰｴﾗｰの表示が...,10,サーバーメンテナンス,利用開始ができない
225,13番車室に入庫するも利用開始操作ができないとのこと。ｾﾝｻｰﾎﾟｰﾙは感知表示になっている...,24,利用開始ができない,利用開始ができない


### exampleデータの出力

In [10]:
cols = ["text", "Original_category" ,"Estimated_category"]
path = "/content/drive/MyDrive/Colab Notebooks/data/output_example.csv"
test[cols].to_csv(path, encoding="cp932")

## 月次データの分類

In [44]:
path = "/content/drive/MyDrive/Colab Notebooks/data/案件一覧_202303.xlsx"
df_op = pd.read_excel(path, sheet_name='案件一覧')

In [45]:
df_op

,受付番号,事業地名,受付日時,大分類,ﾄﾗﾌﾞﾙ種類,入電内容,対応内容,対応結果補足,性別
0,DSP-2303-00001,DSPPPA0035 港区高輪2丁目第1,2023/03/01 06:24,問い合わせ・報告,【DSP】【入電】利用方法問い合わせ,15番車室。入庫するも、ｷｰﾌﾟなしの利用開始操作ができないとの事。,お詫び申し上げ、出庫時に再度お試し頂くようにお伝えし、了承。,NaN,*******
1,DSP-2303-00002,DSPGSN0098 GSｽﾏｰﾄﾊﾟｰｷﾝｸﾞ河原町丸太町,2023/03/01 00:26,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ),ｺﾝｿｰﾙにて【20:13、入庫中】を確認。該当する決済履歴なし。※駆けつけ対応ｴﾘｱ外,ｺﾝｿｰﾙにて出庫中（8:39）を確認。該当する決済履歴はなし。,*******
2,DSP-2303-00003,DSPOSG0082 ｾﾚﾊﾟｰｸ室町高辻町,2023/03/01 00:41,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ),ｺﾝｿｰﾙにて【00:18、入庫中】を確認。該当する決済履歴なし。※駆けつけ対応ｴﾘｱ外,ｺﾝｿｰﾙにて【20:14、出庫中】を確認。該当する決済履歴なし。,*******
3,DSP-2303-00004,DSPPPA0125 世田谷区奥沢4丁目第1,2023/03/01 04:43,場内障害,【DSP】【発報】GW発報,【DSP】GWﾊﾞｯﾃﾘｰ確認依頼1段階目,ｺﾝｿｰﾙにてGWﾊﾞｯﾃﾘｰLV1、残量:8721を確認。ｾﾝｻｰ･ﾊｰﾄﾞｳｪｱｽﾃｰﾀ...,ｺﾝｿｰﾙにてGWﾊﾞｯﾃﾘｰ残量:12475、ｾﾝｻｰ･GWﾊｰﾄﾞｳｪｱｽﾃｰﾀｽともに...,*******
4,DSP-2303-00005,DSPPPA0125 世田谷区奥沢4丁目第1,2023/03/01 05:31,場内障害,【DSP】【発報】GW発報,【DSP】GW稼動停止,案件【DSP-2303-00004】にて対応中。,NaN,*******
...,...,...,...,...,...,...,...,...,...
1462,DSP-2303-01464,DSPOOE0001 ﾀﾞｲｹﾝ西宮ﾊﾟｰｷﾝｸﾞ,2023/03/31 20:56,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t2\t\t\...,ｺﾝｿｰﾙにて20:35入庫中を確認。利用履歴無し。→PAへ点検依頼。,PA対応。現着時、予約専用2番車室に在車有り。CCにて不正車輌を確認。①貼紙実施。 ...,*******
1463,DSP-2303-01465,DSPSSI0019 江東区古石場2丁目第1,2023/03/31 21:26,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t1\t\t\...,ｺﾝｿｰﾙにて21:07入庫中を確認。同時間に出庫履歴(利用開始操作済み)があり、再入庫後に...,NaN,*******
1464,DSP-2304-00001,DSPPPA0167 川崎昭和2丁目第1,2023/03/31 21:41,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t1\t\t\...,ｺﾝｿｰﾙにてｷｰﾌﾟなし利用開始操作を確認。,NaN,*******
1465,DSP-2304-00002,DSPOOE0001 ﾀﾞｲｹﾝ西宮ﾊﾟｰｷﾝｸﾞ,2023/03/31 22:11,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ) 5番_x000D_\n,ｺﾝｿｰﾙにて【21:54】入庫中を確認。予約履歴なし。→PAへ現地確認依頼。,PA対応。現着時、予約専用5番車室に在車有り。CCにて不正車輌を確認。①貼紙実施。 ...,*******


## 企業コードの振り分け

In [46]:
kigyo_mapping = {
    "HTN":"HT",
    "OGS":"OGBC",
    "ITC":"ITC",
    "NLS":"NLS",
    "PJS":"PJ",
    "NBP":"NB",
    "APS":"APS",
    "PRS":"PRS"
    }
def map_kigyo(input_text):
  output_text = ""
  if input_text in kigyo_mapping.keys():
    output_text = kigyo_mapping[input_text]
  else:
    output_text = ""

  return output_text

In [47]:
df_kigyo = df_op[["事業地名"]]
df_kigyo["企業_tmp"] =df_kigyo["事業地名"].str[3:6]
df_kigyo["企業"] = [map_kigyo(text) for text in df_kigyo["企業_tmp"]]
df_op["企業"] = df_kigyo["企業"]
df_op

,受付番号,事業地名,受付日時,大分類,ﾄﾗﾌﾞﾙ種類,入電内容,対応内容,対応結果補足,性別,企業
0,DSP-2303-00001,DSPPPA0035 港区高輪2丁目第1,2023/03/01 06:24,問い合わせ・報告,【DSP】【入電】利用方法問い合わせ,15番車室。入庫するも、ｷｰﾌﾟなしの利用開始操作ができないとの事。,お詫び申し上げ、出庫時に再度お試し頂くようにお伝えし、了承。,NaN,*******,
1,DSP-2303-00002,DSPGSN0098 GSｽﾏｰﾄﾊﾟｰｷﾝｸﾞ河原町丸太町,2023/03/01 00:26,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ),ｺﾝｿｰﾙにて【20:13、入庫中】を確認。該当する決済履歴なし。※駆けつけ対応ｴﾘｱ外,ｺﾝｿｰﾙにて出庫中（8:39）を確認。該当する決済履歴はなし。,*******,
2,DSP-2303-00003,DSPOSG0082 ｾﾚﾊﾟｰｸ室町高辻町,2023/03/01 00:41,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ),ｺﾝｿｰﾙにて【00:18、入庫中】を確認。該当する決済履歴なし。※駆けつけ対応ｴﾘｱ外,ｺﾝｿｰﾙにて【20:14、出庫中】を確認。該当する決済履歴なし。,*******,
3,DSP-2303-00004,DSPPPA0125 世田谷区奥沢4丁目第1,2023/03/01 04:43,場内障害,【DSP】【発報】GW発報,【DSP】GWﾊﾞｯﾃﾘｰ確認依頼1段階目,ｺﾝｿｰﾙにてGWﾊﾞｯﾃﾘｰLV1、残量:8721を確認。ｾﾝｻｰ･ﾊｰﾄﾞｳｪｱｽﾃｰﾀ...,ｺﾝｿｰﾙにてGWﾊﾞｯﾃﾘｰ残量:12475、ｾﾝｻｰ･GWﾊｰﾄﾞｳｪｱｽﾃｰﾀｽともに...,*******,
4,DSP-2303-00005,DSPPPA0125 世田谷区奥沢4丁目第1,2023/03/01 05:31,場内障害,【DSP】【発報】GW発報,【DSP】GW稼動停止,案件【DSP-2303-00004】にて対応中。,NaN,*******,
...,...,...,...,...,...,...,...,...,...,...
1462,DSP-2303-01464,DSPOOE0001 ﾀﾞｲｹﾝ西宮ﾊﾟｰｷﾝｸﾞ,2023/03/31 20:56,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t2\t\t\...,ｺﾝｿｰﾙにて20:35入庫中を確認。利用履歴無し。→PAへ点検依頼。,PA対応。現着時、予約専用2番車室に在車有り。CCにて不正車輌を確認。①貼紙実施。 ...,*******,
1463,DSP-2303-01465,DSPSSI0019 江東区古石場2丁目第1,2023/03/31 21:26,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t1\t\t\...,ｺﾝｿｰﾙにて21:07入庫中を確認。同時間に出庫履歴(利用開始操作済み)があり、再入庫後に...,NaN,*******,
1464,DSP-2304-00001,DSPPPA0167 川崎昭和2丁目第1,2023/03/31 21:41,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t1\t\t\...,ｺﾝｿｰﾙにてｷｰﾌﾟなし利用開始操作を確認。,NaN,*******,
1465,DSP-2304-00002,DSPOOE0001 ﾀﾞｲｹﾝ西宮ﾊﾟｰｷﾝｸﾞ,2023/03/31 22:11,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ) 5番_x000D_\n,ｺﾝｿｰﾙにて【21:54】入庫中を確認。予約履歴なし。→PAへ現地確認依頼。,PA対応。現着時、予約専用5番車室に在車有り。CCにて不正車輌を確認。①貼紙実施。 ...,*******,


In [48]:
df_clf = df_op[df_op["企業"]!=""][["入電内容"]]
df_clf = df_clf.rename(columns={"入電内容":"text"})
df_clf["target"] = 0

In [49]:
clf_data = CustomData(df_clf, tokenizer)
clf_dl = DataLoader(clf_data, batch_size = 32, shuffle = False, drop_last = False)

In [50]:
preds = []
trained_model.eval()
loader = DataLoader(clf_data)
with torch.no_grad():
    for batch in tqdm(clf_dl):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        token_type_ids = batch[2].to(device)
        logits = trained_model(input_ids, attention_mask, token_type_ids)
        preds.append(nn.Softmax(dim = 1)(logits).cpu().numpy())
preds = np.concatenate(preds, axis = 0)
pred_labels = preds.argmax(axis = 1)
pred_labels

100%|██████████| 4/4 [00:01<00:00,  2.46it/s]


array([12, 24, 20, 12, 12,  4, 12, 12, 24, 20, 18, 10, 18, 24, 33, 12, 20,
       26, 12, 27, 24, 24, 20, 20, 20,  4, 20, 18, 18, 12,  4, 12, 26, 20,
       24, 27, 33, 24, 24,  4, 12, 27, 12, 12, 18, 16, 16, 24, 20, 24, 10,
       24, 33,  4, 12,  4, 24, 16, 20, 24, 16, 12, 18, 12, 27, 12, 20, 24,
       18, 20, 12, 27, 24, 27, 16, 24, 24, 20, 17, 24, 20, 18, 18, 27, 26,
       12, 27,  6, 24, 12, 24, 24, 12,  4, 24, 20,  4,  4, 20, 18, 24, 20,
       24, 16, 10,  4,  8, 23, 20,  4, 24])

In [51]:
df_clf["Estimated_category"] = le.inverse_transform(pred_labels)
df_clf

,text,target,Estimated_category
11,3番ｷｰﾌﾟし来場したが別車両在車中。,0,一般ユーザーの入庫
19,4番車室に朝入庫。利用開始操作ができないとの事。【5:44入庫中】確認。,0,利用開始ができない
33,24時間以上利用したい場合は別途手続きは必要か問いあり。,0,利用方法の問い合わせ
48,1番車室をｷｰﾌﾟするも、別車両が入庫中であり、利用不可とのこと。,0,一般ユーザーの入庫
55,ｷｰﾌﾟして来場するも、別の車両が駐車しているとのこと。,0,一般ユーザーの入庫
...,...,...,...
1405,21番車室利用。長期になっているとのﾒｯｾｰｼﾞが届いたとの事。ｺﾝｿｰﾙにて【3/25/1...,0,キープ車室への不正入庫
1421,【DSP】利用時間超過駐車_x000D_\n,0,利用時間超過駐車
1422,2月に3日間ほどｺｲﾝﾊﾟｰｷﾝｸﾞ利用したが、領収書を紛失し発行希望。,0,利用方法の問い合わせ
1435,月極に関する問い合わせ,0,その他（現地精算）


In [54]:
df_op["分類"] = df_clf["Estimated_category"]
df_op["分類"] = df_op["分類"].fillna("")
df_op = df_op[['企業', '分類', '受付番号', '事業地名', '受付日時', '大分類', 'ﾄﾗﾌﾞﾙ種類', '入電内容', '対応内容', '対応結果補足',
       '性別']]
df_op       

,企業,分類,受付番号,事業地名,受付日時,大分類,ﾄﾗﾌﾞﾙ種類,入電内容,対応内容,対応結果補足,性別
0,,,DSP-2303-00001,DSPPPA0035 港区高輪2丁目第1,2023/03/01 06:24,問い合わせ・報告,【DSP】【入電】利用方法問い合わせ,15番車室。入庫するも、ｷｰﾌﾟなしの利用開始操作ができないとの事。,お詫び申し上げ、出庫時に再度お試し頂くようにお伝えし、了承。,NaN,*******
1,,,DSP-2303-00002,DSPGSN0098 GSｽﾏｰﾄﾊﾟｰｷﾝｸﾞ河原町丸太町,2023/03/01 00:26,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ),ｺﾝｿｰﾙにて【20:13、入庫中】を確認。該当する決済履歴なし。※駆けつけ対応ｴﾘｱ外,ｺﾝｿｰﾙにて出庫中（8:39）を確認。該当する決済履歴はなし。,*******
2,,,DSP-2303-00003,DSPOSG0082 ｾﾚﾊﾟｰｸ室町高辻町,2023/03/01 00:41,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ),ｺﾝｿｰﾙにて【00:18、入庫中】を確認。該当する決済履歴なし。※駆けつけ対応ｴﾘｱ外,ｺﾝｿｰﾙにて【20:14、出庫中】を確認。該当する決済履歴なし。,*******
3,,,DSP-2303-00004,DSPPPA0125 世田谷区奥沢4丁目第1,2023/03/01 04:43,場内障害,【DSP】【発報】GW発報,【DSP】GWﾊﾞｯﾃﾘｰ確認依頼1段階目,ｺﾝｿｰﾙにてGWﾊﾞｯﾃﾘｰLV1、残量:8721を確認。ｾﾝｻｰ･ﾊｰﾄﾞｳｪｱｽﾃｰﾀ...,ｺﾝｿｰﾙにてGWﾊﾞｯﾃﾘｰ残量:12475、ｾﾝｻｰ･GWﾊｰﾄﾞｳｪｱｽﾃｰﾀｽともに...,*******
4,,,DSP-2303-00005,DSPPPA0125 世田谷区奥沢4丁目第1,2023/03/01 05:31,場内障害,【DSP】【発報】GW発報,【DSP】GW稼動停止,案件【DSP-2303-00004】にて対応中。,NaN,*******
...,...,...,...,...,...,...,...,...,...,...,...
1462,,,DSP-2303-01464,DSPOOE0001 ﾀﾞｲｹﾝ西宮ﾊﾟｰｷﾝｸﾞ,2023/03/31 20:56,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t2\t\t\...,ｺﾝｿｰﾙにて20:35入庫中を確認。利用履歴無し。→PAへ点検依頼。,PA対応。現着時、予約専用2番車室に在車有り。CCにて不正車輌を確認。①貼紙実施。 ...,*******
1463,,,DSP-2303-01465,DSPSSI0019 江東区古石場2丁目第1,2023/03/31 21:26,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t1\t\t\...,ｺﾝｿｰﾙにて21:07入庫中を確認。同時間に出庫履歴(利用開始操作済み)があり、再入庫後に...,NaN,*******
1464,,,DSP-2304-00001,DSPPPA0167 川崎昭和2丁目第1,2023/03/31 21:41,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】【発報】ｾﾝｻｰ発報\t【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ)\t1\t\t\...,ｺﾝｿｰﾙにてｷｰﾌﾟなし利用開始操作を確認。,NaN,*******
1465,,,DSP-2304-00002,DSPOOE0001 ﾀﾞｲｹﾝ西宮ﾊﾟｰｷﾝｸﾞ,2023/03/31 22:11,場内障害,【DSP】【発報】ｾﾝｻｰ発報,【DSP】ﾁｪｯｸｲﾝなし駐車(予約ﾅｼ) 5番_x000D_\n,ｺﾝｿｰﾙにて【21:54】入庫中を確認。予約履歴なし。→PAへ現地確認依頼。,PA対応。現着時、予約専用5番車室に在車有り。CCにて不正車輌を確認。①貼紙実施。 ...,*******


In [56]:
path = "/content/drive/MyDrive/Colab Notebooks/data/output_peasy.csv"
df_op.to_csv(path, encoding="cp932", index=False)